In [ ]:
import cv2
import numpy as np
import os

class FaceDetector:
    def __init__(self):
        # Load the pre-trained Haar Cascade Classifier
        # Note: You'll need to download this XML file from OpenCV's github
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        
    def detect_faces(self, image_path, show_result=True):
        # Read the image
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Detect faces in the image
        # Parameters explanation:
        # scaleFactor: How much the image size is reduced at each image scale
        # minNeighbors: How many neighbors each candidate rectangle should have
        faces = self.face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30)
        )
        
        # Draw rectangles around detected faces
        for (x, y, w, h) in faces:
            cv2.rectangle(
                image, 
                (x, y),                    # Top-left corner
                (x + w, y + h),            # Bottom-right corner
                (255, 0, 0),               # Color (BGR)
                2                          # Thickness
            )
        
        if show_result:
            # Display the result
            cv2.imshow('Detected Faces', image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        
        return faces, image

    def extract_face(self, image_path, output_size=(128, 128)):
        """Extract, resize and normalize detected faces."""
        faces, image = self.detect_faces(image_path, show_result=False)
        face_images = []
        
        for (x, y, w, h) in faces:
            # Extract the face region
            face = image[y:y+h, x:x+w]
            
            # Resize to standard size
            face = cv2.resize(face, output_size)
            
            # Convert to grayscale
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            
            # Normalize pixel values
            face = face / 255.0
            
            face_images.append(face)
            
        return face_images

def process_dataset(data_dir, output_dir):
    """Process all images in a directory and save extracted faces."""
    detector = FaceDetector()
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Process each image in the directory
    for image_name in os.listdir(data_dir):
        if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(data_dir, image_name)
            
            try:
                # Extract faces
                faces = detector.extract_face(image_path)
                
                # Save each detected face
                for i, face in enumerate(faces):
                    output_path = os.path.join(
                        output_dir,
                        f"{os.path.splitext(image_name)[0]}_face_{i}.jpg"
                    )
                    # Convert back to 0-255 range for saving
                    face_img = (face * 255).astype(np.uint8)
                    cv2.imwrite(output_path, face_img)
                    
            except Exception as e:
                print(f"Error processing {image_name}: {str(e)}")

# Example usage
if __name__ == "__main__":
    # Test single image detection
    detector = FaceDetector()
    detector.detect_faces("C:/Users/teddy/OneDrive/Documents/Documents/Cst-440/NormalPeopleFaces/1 (1).jpeg")
    
    # Process entire dataset
    process_dataset(
        data_dir="Donald Trump",
        output_dir="processed_images/trump"
    )
    process_dataset(
        data_dir="NormalPeopleFaces",
        output_dir="processed_images/non_trump"
    )

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'raw_images/trump'